# We need bigwig files for fast access to coverage (used by the score_edits_* scripts)
- Used when computing edit/C metric
- this notebook assumes a reverse stranded SE library. Otherwise you may need to reverse the ```-strand +``` param

In [6]:
import glob
import os
from qtools import Submitter
from tqdm import tnrange, tqdm_notebook

In [7]:
input_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data/final_analysis/01_SAILOR_bulk_rnaseq/outputs'
output_dir = '/home/bay001/projects/kris_apobec_20200121/permanent_data/final_analysis/01_SAILOR_bulk_rnaseq/outputs/combined_outputs_w_cov_info'

### inputs should be the two ```*.sorted.rmdup.readfiltered.bam``` files that are intermediates from SAILOR. These are BAM files, split into their respective strands, and filtered for any low-quality or unexpected (non C/T) reads.

In [8]:
all_fwd_bams = glob.glob(os.path.join(input_dir, '*/results/*.fwd.sorted.rmdup.readfiltered.bam'))
print(len(all_fwd_bams))
all_fwd_bams[:3]

131


['/home/bay001/projects/kris_apobec_20200121/permanent_data/final_analysis/01_SAILOR_bulk_rnaseq/outputs/SLBP_1000_48hr_B_merged_R1_a0_b0_e0.01/results/SLBP_1000_48hr_B_merged_R1.fastqTr.sorted.STARUnmapped.out.sorted.STARAligned.out.sorted_a0_b0_e0.01.fwd.sorted.rmdup.readfiltered.bam',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data/final_analysis/01_SAILOR_bulk_rnaseq/outputs/Sample8_S28_merged_a0_b0_e0.01/results/Sample8_S28_merged.fastqTr.sorted.STARUnmapped.out.sorted.STARAligned.out.sorted_a0_b0_e0.01.fwd.sorted.rmdup.readfiltered.bam',
 '/home/bay001/projects/kris_apobec_20200121/permanent_data/final_analysis/01_SAILOR_bulk_rnaseq/outputs/RBFOX2-0_S6_L001_R1_001_a0_b0_e0.01/results/RBFOX2-0_S6_L001_R1_001.fastqTr.sorted.STARUnmapped.out.sorted.STARAligned.out.sorted_a0_b0_e0.01.fwd.sorted.rmdup.readfiltered.bam']

# Write the bam to bigwig commands

In [9]:
chrom_sizes_file = '/projects/ps-yeolab3/bay001/annotations/hg19/hg19.chrom.sizes'
cmds = []
for fwd_bam in all_fwd_bams:
    fwd_bg = os.path.join(output_dir, os.path.splitext(os.path.basename(fwd_bam))[0] + ".bg")
    fwd_sorted_bg = os.path.join(output_dir, os.path.splitext(os.path.basename(fwd_bam))[0] + ".sorted.bg")
    fwd_sorted_bw = os.path.join(output_dir, os.path.splitext(os.path.basename(fwd_bam))[0] + ".sorted.bw")
    if not os.path.exists(fwd_sorted_bw):
        cmd = 'module load makebigwigfiles;'
        cmd += 'bedtools genomecov -split -strand - -g {} -bg -ibam {} > {};'.format(chrom_sizes_file, fwd_bam, fwd_bg)
        cmd += 'bedtools sort -i {} > {};'.format(fwd_bg, fwd_sorted_bg)
        cmd += 'bedGraphToBigWig {} {} {};'.format(fwd_sorted_bg, chrom_sizes_file, fwd_sorted_bw)
        cmds.append(cmd)
    
for fwd_bam in all_fwd_bams:
    rev_bam = fwd_bam.replace('.fwd.sorted.rmdup.readfiltered.bam', '.rev.sorted.rmdup.readfiltered.bam')
    rev_bg = os.path.join(output_dir, os.path.splitext(os.path.basename(rev_bam))[0] + ".bg")
    rev_sorted_bg = os.path.join(output_dir, os.path.splitext(os.path.basename(rev_bam))[0] + ".sorted.bg")
    rev_sorted_bw = os.path.join(output_dir, os.path.splitext(os.path.basename(rev_bam))[0] + ".sorted.bw")
    if not os.path.exists(rev_sorted_bw):
        cmd = 'module load makebigwigfiles;'
        cmd += 'bedtools genomecov -split -strand + -g {} -bg -ibam {} > {};'.format(chrom_sizes_file, rev_bam, rev_bg)
        cmd += 'bedtools sort -i {} > {};'.format(rev_bg, rev_sorted_bg)
        cmd += 'bedGraphToBigWig {} {} {};'.format(rev_sorted_bg, chrom_sizes_file, rev_sorted_bw)
        cmds.append(cmd)
print(len(cmds))

0


In [10]:
Submitter(commands=cmds, job_name='bamToBigWig', array=True, nodes=1, ppn=1, submit=True, walltime='8:00:00')

Writing 0 tasks as an array-job.
Wrote commands to bamToBigWig.sh.


CalledProcessError: Command '['qsub', 'bamToBigWig.sh']' returned non-zero exit status 235

# 